<a href="https://colab.research.google.com/github/jiho051225/footballdata6week/blob/main/xt/xT_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 새 섹션

In [ ]:
!git clone https://github.com/narame7/UOS-FootballDataAnalytics-Tutorial

In [ ]:
%cd 'UOS-FootballDataAnalytics-Tutorial/Week 8'

In [ ]:
!pip install pandera==0.13.4 scipy==1.11.4 statsbombpy

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings(action="ignore", message="credentials were not supplied. open data access only")

from tqdm.notebook import tqdm

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import matplotsoccer as mps # 축구 이벤트 데이터 시각화
from socceraction.data.statsbomb import StatsBombLoader # StatsBomb 데이터 로더
import socceraction.spadl as spadl # SPADL 포맷 변환 및 처리
import socceraction.xthreat as xthreat # Expected Threat (xT) 계산

 Set up the StatsBombLoader

In [ ]:
# StatsBomb 데이터 로더 객체를 생성합니다. 무료 공개 데이터 접근을 위해 creds를 None으로 설정합니다.
SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})
SBL

In [ ]:
# 사용 가능한 대회 목록
competitions = SBL.competitions()
competitions

In [ ]:
game_counts = []

for index, row in competitions.iterrows():
    comp_id = row['competition_id']
    season_id = row['season_id']
    comp_name = row['competition_name']
    season_name = row['season_name']

    # Fetch games for the current competition and season
    games_for_comp = SBL.games(comp_id, season_id)
    total_games = len(games_for_comp)

    game_counts.append({
        'competition_name': comp_name,
        'season_name': season_name,
        'total_games': total_games
    })

df_game_counts = pd.DataFrame(game_counts)
display(df_game_counts)

In [ ]:
# 'FIFA World Cup' 2018 시즌
selected_competitions = competitions[
	(competitions.competition_name == "FIFA World Cup")
	& (competitions.season_name == "2018")
]
selected_competitions

In [ ]:
games = pd.concat([
	SBL.games(row.competition_id, row.season_id)
	for row in selected_competitions.itertuples()
])
games[["home_team_id", "away_team_id", "game_date", "home_score", "away_score"]]

🥅 SPADL (Soccer Player Action Description Language) 전처리 및 로딩
SPADL: StatsBomb, Opta, Wyscout 등 다양한 데이터 제공 업체의 서로 다른 이벤트 데이터 포맷을 통일된 액션 기반 포맷으로 변환하는 언어입니다.
현대 축구 데이터 분석 연구에서는 SPADL이라는 통일된 포맷을 사용하여 분석하는 연구가 주를 이룹니다.
주요 정보: 액션 유형, 시작/종료 좌표, 선수, 팀, 결과 등

In [ ]:
# 모든 액션이 왼쪽에서 오른쪽으로 진행되도록 좌표를 변환하는 함수입니다.
# xT 모델은 공격 방향을 통일해야 합니다.
def play_left_to_right(actions: pd.DataFrame, home_team_id: int) -> pd.DataFrame:
	"""
		모든 액션이 동일한 경기 방향(왼쪽에서 오른쪽)으로 수행되도록 변환합니다.

		이 함수는 각 액션의 시작 및 종료 위치를 변경하여,
		마치 첫 액션을 수행한 팀이 항상 왼쪽에서 오른쪽으로 플레이하는 것처럼 만듭니다.

		Parameters
		----------
		actions : pd.DataFrame
			한 경기의 액션 데이터프레임.
		home_team_id : int
			홈팀의 ID.

		Returns
		-------
		pd.DataFrame
			모든 액션이 왼쪽에서 오른쪽으로 수행되도록 좌표가 변환된 데이터프레임.
	"""

	away_idx = actions.team_id != home_team_id

	for col in ["start_x", "end_x"]:
		actions.loc[away_idx, col] = spadl.config.field_length - actions[away_idx][col].values
	for col in ["start_y", "end_y"]:
		actions.loc[away_idx, col] = spadl.config.field_width - actions[away_idx][col].values

	return actions

In [ ]:
games_verbose = tqdm(list(games.itertuples()), desc="Loading game data")
df_teams, df_players, df_actions = [], [], []

for game in games_verbose:
	# load data
	df_teams.append(SBL.teams(game.game_id))
	df_players.append(SBL.players(game.game_id))
	events = SBL.events(game.game_id)

	# StatsBomb 이벤트 데이터를 SPADL 액션 데이터로 변환합니다.
	actions = spadl.statsbomb.convert_to_actions(
			events,
			home_team_id=game.home_team_id,
			xy_fidelity_version=1,
			shot_fidelity_version=1
	)
	actions = play_left_to_right(actions, home_team_id=game.home_team_id) # xT 계산을 위해 모든 액션 방향을 왼쪽에서 오른쪽으로 통일합니다.
	df_actions.append(actions)

df_teams = pd.concat(df_teams).drop_duplicates(subset="team_id")
df_player_games = pd.concat(df_players).reset_index(drop=True) # 경기 별 선수 정보. 경기 별 선수 정보 (position, minutes_played)은 다름)
df_players = pd.concat(df_players).drop_duplicates(subset="player_id") # 선수 정보. 선수 정보 (name, nationality 등)은 동일)
df_actions = pd.concat(df_actions).reset_index(drop=True)
df_actions.head()

In [ ]:
df_teams

In [ ]:
df_player_games

In [ ]:
df_players

SPADL 데이터 시각화
matplotsoccer: 축구 이벤트/액션 데이터를 경기장 위에 시각화하는 라이브러리.

In [ ]:
def nice_time(row):
	minute = int((row.period_id-1)*45 +row.time_seconds // 60)
	second = int(row.time_seconds % 60)
	return f"{minute}m{second}s"

def plot_actions(a: pd.DataFrame, g: pd.Series) -> None:
	home_team_name = df_teams[df_teams.team_id == g.home_team_id].team_name.values[0]
	away_team_name = df_teams[df_teams.team_id == g.away_team_id].team_name.values[0]

	minute = int((a.period_id.values[0]-1) * 45 + a.time_seconds.values[0] // 60)
	game_info = f"{g.game_date} {home_team_name} {g.home_score}-{g.away_score} {away_team_name} {minute + 1}'"
	print(game_info)

	a["player_name"] = a.player_id.map(df_players.set_index("player_id").player_name)
	a["team_name"] = a.team_id.map(df_teams.set_index("team_id").team_name)
	a["type_name"] = a.type_id.map(spadl.config.actiontypes_df().type_name.to_dict())
	a["result_name"] = a.result_id.map(spadl.config.results_df().result_name.to_dict())
	a["nice_time"] = a.apply(nice_time, axis=1)

	if "xT_value" in a.columns:
		labels = a[["nice_time", "type_name", "player_name", "team_name", "xT_value"]]
		labeltitle = ["time", "actiontype", "player", "team", "xT_value"]
	else:
		labels = a[["nice_time", "type_name", "player_name", "team_name"]]
		labeltitle = ["time", "actiontype", "player", "team"]

	away_idx = a.team_id != g.home_team_id
	for col in ["start_x", "end_x"]:
		a.loc[away_idx, col] = spadl.config.field_length - a[away_idx][col].values
	for col in ["start_y", "end_y"]:
		a.loc[away_idx, col] = spadl.config.field_width - a[away_idx][col].values

	mps.actions(
		location=a[["start_x", "start_y", "end_x", "end_y"]],
		action_type=a.type_name,
		team= a.team_name,
		result= a.result_name == "success",
		label=labels,
		labeltitle=labeltitle,
		zoom=False,
		figsize=6,
		color="green"
	)

In [ ]:
# Select the 5 actions preceding the 2-0
shot = 5
a = df_actions[shot-5:shot+2].copy()

# Print the game date and timestamp of the goal
game_id = 7559
g = games[games.game_id == game_id].iloc[0]

plot_actions(a, g)
a

Expected Threat (xT)
Expected Threat 참고 자료
xT 계산에 사용되는 액션: pass(cross), dribble, shot(goal)

In [ ]:
df_actions

In [ ]:
A = df_actions.copy()
A["type_name"] = A.type_id.map(spadl.config.actiontypes_df().type_name.to_dict())
A["result_name"] = A.result_id.map(spadl.config.results_df().result_name.to_dict())
A["bodypart_name"] = A.bodypart_id.map(spadl.config.bodyparts_df().bodypart_name.to_dict())

A.head()

In [ ]:
df_actions = df_actions.merge(
    games[["game_id", "home_team_id", "away_team_id", "home_score", "away_score", "game_date"]],
    on="game_id",
    how="left"
)
print("Merged df_actions with game information. Displaying head of the updated df_actions:")
df_actions.head()

# Task
경기 정보(스코어, 시간)와 액션 데이터를 결합하여 각 액션이 발생한 시점의 경기 상황(전반/후반, 리드/추격)을 식별하고, 전반과 후반으로 나누어 각 시간대별 xT 생성량의 변화를 분석하고 시각화합니다. 또한, 리드, 동점, 추격 상황별로 데이터를 분류하여 각 상황에서 발생하는 xT의 패턴을 분석하고 시각화합니다.

In [ ]:
df_actions["type_name"] = df_actions.type_id.map(spadl.config.actiontypes_df().type_name.to_dict())
df_actions["result_name"] = df_actions.result_id.map(spadl.config.results_df().result_name.to_dict())

print("Added 'type_name' and 'result_name' to df_actions. Displaying head:")
df_actions.head()

In [ ]:
df_actions = df_actions.sort_values(by=["game_id", "period_id", "time_seconds"]).reset_index(drop=True)

def calculate_current_scores(df_game_actions):
    current_home_score = 0
    current_away_score = 0
    scores = []

    if 'home_team_id_x' in df_game_actions.columns and 'away_team_id_x' in df_game_actions.columns:
        game_home_team_id = df_game_actions['home_team_id_x'].iloc[0]
        game_away_team_id = df_game_actions['away_team_id_x'].iloc[0]
    elif 'home_team_id' in df_game_actions.columns and 'away_team_id' in df_game_actions.columns:
        game_home_team_id = df_game_actions['home_team_id'].iloc[0]
        game_away_team_id = df_game_actions['away_team_id'].iloc[0]
    else:
        print(f"Warning: neither home_team_id_x/away_team_id_x nor home_team_id/away_team_id found in df_game_actions for game_id {df_game_actions['game_id'].iloc[0]}")
        return pd.DataFrame(scores, columns=['current_home_score', 'current_away_score'], index=df_game_actions.index)

    for idx, row in df_game_actions.iterrows():
        scores.append((current_home_score, current_away_score))
        if row.type_name == 'shot' and row.result_name == 'success':
            if row.team_id == game_home_team_id:
                current_home_score += 1
            elif row.team_id == game_away_team_id:
                current_away_score += 1
    return pd.DataFrame(scores, columns=['current_home_score', 'current_away_score'], index=df_game_actions.index)


current_scores_df = df_actions.groupby('game_id', group_keys=False).apply(calculate_current_scores)

# Drop any existing 'current_home_score' and 'current_away_score' columns to prevent duplicates
df_actions = df_actions.loc[:, ~df_actions.columns.duplicated()].copy()
df_actions = df_actions.drop(columns=[col for col in ['current_home_score', 'current_away_score'] if col in df_actions.columns], errors='ignore')

df_actions = pd.concat([df_actions, current_scores_df], axis=1)

print("Calculated current scores for each action. Displaying head of updated df_actions:")
df_actions.head()

In [ ]:
df_actions['half'] = df_actions['period_id'].apply(lambda x: 'first_half' if x == 1 else 'second_half')

def calculate_score_diff(row):
    # Use the suffixed column names present in df_actions
    if row['team_id'] == row['home_team_id']:
        return row['current_home_score'] - row['current_away_score']
    else:
        return row['current_away_score'] - row['current_home_score']

df_actions['score_diff'] = df_actions.apply(calculate_score_diff, axis=1)

def get_game_situation(score_diff):
    if score_diff > 0:
        return 'leading'
    elif score_diff == 0:
        return 'tied'
    else:
        return 'trailing'

df_actions['game_situation'] = df_actions['score_diff'].apply(get_game_situation)

print("Added 'half', 'score_diff', and 'game_situation' to df_actions. Displaying head:")
df_actions.head()

## 전반/후반 xT 변화 분석

### Subtask:
전반과 후반으로 데이터를 나누어 각 시간대별 xT 생성량의 변화를 분석하고 시각화합니다.


In [ ]:
# Before merging, drop any existing columns that would conflict with the 'xT_value' column from xT_values.
# This includes 'xT_value' itself, and potential suffixed versions from previous merges.
columns_to_drop = [col for col in df_actions.columns if 'xT_value' in col]
df_actions = df_actions.drop(columns=columns_to_drop, errors='ignore')

xT_values = mov_actions[['game_id', 'original_event_id', 'period_id', 'time_seconds', 'xT_value']]
df_actions = df_actions.merge(xT_values, on=['game_id', 'original_event_id', 'period_id', 'time_seconds'], how='left')
df_actions['xT_value'] = df_actions['xT_value'].fillna(0)

print("Merged xT values into df_actions and filled NaN with 0. Displaying head:")
df_actions.head()

In [ ]:
mov_actions = xthreat.get_successful_move_actions(df_actions)
mov_actions["xT_value"] = xTModel.rate(mov_actions)
xT_values = mov_actions[['game_id', 'original_event_id', 'period_id', 'time_seconds', 'xT_value']]
df_actions = df_actions.merge(xT_values, on=['game_id', 'original_event_id', 'period_id', 'time_seconds'], how='left')
df_actions['xT_value'] = df_actions['xT_value'].fillna(0)

print("Merged xT values into df_actions and filled NaN with 0. Displaying head:")
df_actions.head()

In [ ]:
xTModel = xthreat.ExpectedThreat(l=16, w=12)
xTModel.fit(df_actions)

mov_actions = xthreat.get_successful_move_actions(df_actions)
mov_actions["xT_value"] = xTModel.rate(mov_actions)
xT_values = mov_actions[['game_id', 'original_event_id', 'period_id', 'time_seconds', 'xT_value']]

# Drop any existing xT_value columns (including suffixed ones) from df_actions before merging
columns_to_drop_xt = [col for col in df_actions.columns if 'xT_value' in col]
df_actions = df_actions.drop(columns=columns_to_drop_xt, errors='ignore')

df_actions = df_actions.merge(xT_values, on=['game_id', 'original_event_id', 'period_id', 'time_seconds'], how='left')
df_actions['xT_value'] = df_actions['xT_value'].fillna(0)

print("Merged xT values into df_actions and filled NaN with 0. Displaying head:")
df_actions.head()

In [ ]:
first_half_actions = df_actions[df_actions['period_id'] == 1].copy()
second_half_actions = df_actions[df_actions['period_id'] >= 2].copy()

def calculate_binned_xt(df, bin_size_minutes=5):
    df['time_bin'] = (df['time_seconds'] / (bin_size_minutes * 60)).astype(int) * bin_size_minutes
    binned_xt = df.groupby(['game_id', 'time_bin'])['xT_value'].sum().reset_index()
    cumulative_binned_xt = binned_xt.groupby('time_bin')['xT_value'].mean().cumsum().reset_index()
    return cumulative_binned_xt

first_half_binned_xt = calculate_binned_xt(first_half_actions)
second_half_binned_xt = calculate_binned_xt(second_half_actions)

plt.figure(figsize=(12, 7))
plt.plot(first_half_binned_xt['time_bin'], first_half_binned_xt['xT_value'], label='First Half xT', marker='o')
plt.plot(second_half_binned_xt['time_bin'], second_half_binned_xt['xT_value'], label='Second Half xT', marker='o')

plt.title('Cumulative Expected Threat (xT) Generation by Half Over Time')
plt.xlabel('Time in Half (minutes)')
plt.ylabel('Cumulative Average xT')
plt.grid(True)
plt.legend()
plt.show()

print("Visualized cumulative xT generation for first and second halves.")

## 리드/추격 상황별 xT 생성 패턴 분석(18,22 월드컵)

리드, 동점, 추격 상황별로 데이터를 분류하고 각 상황에서 발생하는 xT의 패턴을 분석하고 시각화


In [ ]:
selected_competitions = competitions[
	(competitions.competition_name == "FIFA World Cup")
	& (competitions.season_name.isin(["2018", "2022"]))
]
print("Filtered competitions for FIFA World Cup 2018 and 2022:")
display(selected_competitions)

In [ ]:
games = pd.concat([
	SBL.games(row.competition_id, row.season_id)
	for row in selected_competitions.itertuples()
])
print("Loaded games for selected competitions. Displaying head with key columns:")
games[["home_team_id", "away_team_id", "game_date", "home_score", "away_score"]].head()

In [ ]:
games_verbose = tqdm(list(games.itertuples()), desc="Loading game data for selected World Cups")
df_teams, df_players, df_actions = [], [], []

for game in games_verbose:
	# load data
	df_teams.append(SBL.teams(game.game_id))
	df_players.append(SBL.players(game.game_id))
	events = SBL.events(game.game_id)

	# StatsBomb 이벤트 데이터를 SPADL 액션 데이터로 변환합니다.
	actions = spadl.statsbomb.convert_to_actions(
		events,
		home_team_id=game.home_team_id,
		xy_fidelity_version=1,
		shot_fidelity_version=1
	)
	actions = play_left_to_right(actions, home_team_id=game.home_team_id) # xT 계산을 위해 모든 액션 방향을 왼쪽에서 오른쪽으로 통일합니다.
	df_actions.append(actions)

df_teams = pd.concat(df_teams).drop_duplicates(subset="team_id")
df_player_games = pd.concat(df_players).reset_index(drop=True) # 경기 별 선수 정보. 경기 별 선수 정보 (position, minutes_played)은 다름)
df_players = pd.concat(df_players).drop_duplicates(subset="player_id") # 선수 정보. 선수 정보 (name, nationality 등)은 동일)
df_actions = pd.concat(df_actions).reset_index(drop=True)

print("Reloaded and converted actions for selected games. Displaying head of df_actions:")
df_actions.head()

In [ ]:
df_actions["type_name"] = df_actions.type_id.map(spadl.config.actiontypes_df().type_name.to_dict())
df_actions["result_name"] = df_actions.result_id.map(spadl.config.results_df().result_name.to_dict())

print("Added 'type_name' and 'result_name' to df_actions. Displaying head:")
df_actions.head()

In [ ]:
df_actions = df_actions.merge(
    games[["game_id", "home_team_id", "away_team_id", "home_score", "away_score", "game_date"]],
    on="game_id",
    how="left"
)

df_actions = df_actions.sort_values(by=["game_id", "period_id", "time_seconds"]).reset_index(drop=True)

def calculate_current_scores(df_game_actions):
    current_home_score = 0
    current_away_score = 0
    scores = []

    # Ensure we use the correct team IDs, handling potential _x suffixes from previous merges if any
    game_home_team_id = df_game_actions['home_team_id'].iloc[0]
    game_away_team_id = df_game_actions['away_team_id'].iloc[0]

    for idx, row in df_game_actions.iterrows():
        scores.append((current_home_score, current_away_score))
        if row.type_name == 'shot' and row.result_name == 'success':
            if row.team_id == game_home_team_id:
                current_home_score += 1
            elif row.team_id == game_away_team_id:
                current_away_score += 1
    return pd.DataFrame(scores, columns=['current_home_score', 'current_away_score'], index=df_game_actions.index)


current_scores_df = df_actions.groupby('game_id', group_keys=False).apply(calculate_current_scores)

# Drop any existing 'current_home_score' and 'current_away_score' columns to prevent duplicates or conflicts
df_actions = df_actions.drop(columns=[col for col in ['current_home_score', 'current_away_score'] if col in df_actions.columns], errors='ignore')

df_actions = pd.concat([df_actions, current_scores_df], axis=1)

print("Recalculated current scores for each action. Displaying head of updated df_actions:")
df_actions.head()

In [ ]:
df_actions = df_actions.merge(
    games[["game_id", "home_team_id", "away_team_id", "home_score", "away_score", "game_date"]],
    on="game_id",
    how="left"
)

df_actions = df_actions.sort_values(by=["game_id", "period_id", "time_seconds"]).reset_index(drop=True)

def calculate_current_scores(df_game_actions):
    current_home_score = 0
    current_away_score = 0
    scores = []

    # Ensure we use the correct team IDs, handling potential _x suffixes from previous merges if any
    game_home_team_id = df_game_actions['home_team_id'].iloc[0]
    game_away_team_id = df_game_actions['away_team_id'].iloc[0]

    for idx, row in df_game_actions.iterrows():
        scores.append((current_home_score, current_away_score))
        if row.type_name == 'shot' and row.result_name == 'success':
            if row.team_id == game_home_team_id:
                current_home_score += 1
            elif row.team_id == game_away_team_id:
                current_away_score += 1
    return pd.DataFrame(scores, columns=['current_home_score', 'current_away_score'], index=df_game_actions.index)


current_scores_df = df_actions.groupby('game_id', group_keys=False).apply(calculate_current_scores, include_groups=False)

# Drop any existing 'current_home_score' and 'current_away_score' columns to prevent duplicates or conflicts
df_actions = df_actions.drop(columns=[col for col in ['current_home_score', 'current_away_score'] if col in df_actions.columns], errors='ignore')

df_actions = pd.concat([df_actions, current_scores_df], axis=1)

print("Recalculated current scores for each action. Displaying head of updated df_actions:")
df_actions.head()


In [ ]:
cols_to_drop_pre_merge = ["home_team_id", "away_team_id", "home_score", "away_score", "game_date"]
df_actions = df_actions.drop(columns=[col for col in cols_to_drop_pre_merge if col in df_actions.columns], errors='ignore')

df_actions = df_actions.merge(
    games[["game_id", "home_team_id", "away_team_id", "home_score", "away_score", "game_date"]],
    on="game_id",
    how="left"
)

df_actions = df_actions.sort_values(by=["game_id", "period_id", "time_seconds"]).reset_index(drop=True)

def calculate_current_scores(df_game_actions):
    current_home_score = 0
    current_away_score = 0
    scores = []

    # Ensure we use the correct team IDs, handling potential _x suffixes from previous merges if any
    game_home_team_id = df_game_actions['home_team_id'].iloc[0]
    game_away_team_id = df_game_actions['away_team_id'].iloc[0]

    for idx, row in df_game_actions.iterrows():
        scores.append((current_home_score, current_away_score))
        if row.type_name == 'shot' and row.result_name == 'success':
            if row.team_id == game_home_team_id:
                current_home_score += 1
            elif row.team_id == game_away_team_id:
                current_away_score += 1
    return pd.DataFrame(scores, columns=['current_home_score', 'current_away_score'], index=df_game_actions.index)


current_scores_df = df_actions.groupby('game_id', group_keys=False).apply(calculate_current_scores, include_groups=False)

# Drop any existing 'current_home_score' and 'current_away_score' columns to prevent duplicates or conflicts
df_actions = df_actions.drop(columns=[col for col in ['current_home_score', 'current_away_score'] if col in df_actions.columns], errors='ignore')

df_actions = pd.concat([df_actions, current_scores_df], axis=1)

print("Recalculated current scores for each action. Displaying head of updated df_actions:")
df_actions.head()

In [ ]:
cols_to_clear = [col for col in df_actions.columns if 'home_team_id' in col or 'away_team_id' in col or 'home_score' in col or 'away_score' in col or 'game_date' in col]
df_actions = df_actions.drop(columns=cols_to_clear, errors='ignore')

# Now, merge with the clean game information
df_actions = df_actions.merge(
    games[["game_id", "home_team_id", "away_team_id", "home_score", "away_score", "game_date"]],
    on="game_id",
    how="left"
)

df_actions = df_actions.sort_values(by=["game_id", "period_id", "time_seconds"]).reset_index(drop=True)

def calculate_current_scores(df_game_actions):
    current_home_score = 0
    current_away_score = 0
    scores = []

    game_home_team_id = df_game_actions['home_team_id'].iloc[0]
    game_away_team_id = df_game_actions['away_team_id'].iloc[0]

    for idx, row in df_game_actions.iterrows():
        scores.append((current_home_score, current_away_score))
        if row.type_name == 'shot' and row.result_name == 'success':
            if row.team_id == game_home_team_id:
                current_home_score += 1
            elif row.team_id == game_away_team_id:
                current_away_score += 1
    return pd.DataFrame(scores, columns=['current_home_score', 'current_away_score'], index=df_game_actions.index)


current_scores_df = df_actions.groupby('game_id', group_keys=False).apply(calculate_current_scores, include_groups=False)

# Drop any existing 'current_home_score' and 'current_away_score' columns to prevent duplicates or conflicts
df_actions = df_actions.drop(columns=[col for col in ['current_home_score', 'current_away_score'] if col in df_actions.columns], errors='ignore')

df_actions = pd.concat([df_actions, current_scores_df], axis=1)

print("Recalculated current scores for each action. Displaying head of updated df_actions:")
df_actions.head()

In [ ]:
df_actions['half'] = df_actions['period_id'].apply(lambda x: 'first_half' if x == 1 else 'second_half')

def calculate_score_diff(row):
    if row['team_id'] == row['home_team_id']:
        return row['current_home_score'] - row['current_away_score']
    else:
        return row['current_away_score'] - row['current_home_score']

df_actions['score_diff'] = df_actions.apply(calculate_score_diff, axis=1)

def get_game_situation(score_diff):
    if score_diff > 0:
        return 'leading'
    elif score_diff == 0:
        return 'tied'
    else:
        return 'trailing'

df_actions['game_situation'] = df_actions['score_diff'].apply(get_game_situation)

print("Added 'half', 'score_diff', and 'game_situation' to df_actions. Displaying head:")
df_actions.head()

In [ ]:
def calculate_continuous_binned_xt(df, bin_size_minutes=5):
    df_copy = df.copy()
    # Calculate continuous game time in seconds
    df_copy['continuous_time'] = (df_copy['period_id'] - 1) * 45 * 60 + df_copy['time_seconds']

    # Filter out times beyond 90 minutes (5400 seconds)
    df_copy = df_copy[df_copy['continuous_time'] <= 90 * 60]

    # Create time bins based on continuous time in minutes
    df_copy['time_bin'] = (df_copy['continuous_time'] / 60 / bin_size_minutes).astype(int) * bin_size_minutes

    # Group by game_id and time_bin to sum xT_value
    binned_xt = df_copy.groupby(['game_id', 'time_bin'])['xT_value'].sum().reset_index()

    # Calculate the cumulative mean of xT_value for each time_bin across all games
    cumulative_binned_xt = binned_xt.groupby('time_bin')['xT_value'].mean().cumsum().reset_index()

    return cumulative_binned_xt

print("Modified `calculate_continuous_binned_xt` function to handle continuous game time and limit to 90 minutes.")

In [ ]:
leading_binned_xt_continuous = calculate_continuous_binned_xt(leading_actions)
tied_binned_xt_continuous = calculate_continuous_binned_xt(tied_actions)
trailing_binned_xt_continuous = calculate_continuous_binned_xt(trailing_actions)

print("Calculated continuous binned xT for 'leading', 'tied', and 'trailing' situations.")
print("Leading situations continuous xT head:")
print(leading_binned_xt_continuous.head())
print("\nTied situations continuous xT head:")
print(tied_binned_xt_continuous.head())
print("\nTrailing situations continuous xT head:")
print(trailing_binned_xt_continuous.head())

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(leading_binned_xt_continuous['time_bin'], leading_binned_xt_continuous['xT_value'], label='Leading xT', marker='o')
plt.plot(tied_binned_xt_continuous['time_bin'], tied_binned_xt_continuous['xT_value'], label='Tied xT', marker='o')
plt.plot(trailing_binned_xt_continuous['time_bin'], trailing_binned_xt_continuous['xT_value'], label='Trailing xT', marker='o')

plt.title('Cumulative Expected Threat (xT) Generation by Game Situation Over Continuous Time (0-90 mins)')
plt.xlabel('Continuous Game Time (minutes)')
plt.ylabel('Cumulative Average xT')
plt.xlim(0, 90) # Explicitly set x-axis limit to 0-90 minutes
plt.grid(True)
plt.legend()
plt.show()

print("Visualized cumulative xT generation for leading, tied, and trailing game situations over continuous time, limited to 0-90 minutes.")

# Task
Add a new column to `df_actions` that categorizes each pass as 'forward', 'backward', or 'sideways' based on the change in its 'start_x' and 'end_x' coordinates.

## Define Forward Passes

### Subtask:
Add a new column to df_actions to categorize passes as 'forward', 'backward', or 'sideways' based on changes in their 'start_x' and 'end_x' coordinates.


**Reasoning**:
First, I will filter `df_actions` to include only 'pass' actions, store it in `df_passes`, calculate the change in x-coordinate (`dx`), and then categorize the passes as 'forward', 'backward', or 'sideways' in a new column called 'pass_direction'.



In [ ]:
df_passes = df_actions[df_actions['type_name'] == 'pass'].copy()
df_passes['dx'] = df_passes['end_x'] - df_passes['start_x']

def get_pass_direction(dx):
    if dx > 0:
        return 'forward'
    elif dx < 0:
        return 'backward'
    else:
        return 'sideways'

df_passes['pass_direction'] = df_passes['dx'].apply(get_pass_direction)

print("Filtered passes, calculated 'dx', and determined 'pass_direction'. Displaying head of df_passes:")
df_passes.head()

**Reasoning**:
Now that 'pass_direction' has been calculated for passes in `df_passes`, I will merge this information back into the original `df_actions` DataFrame and fill any resulting NaN values for non-pass actions with 'not_a_pass'.



In [ ]:
df_actions = df_actions.merge(
    df_passes[['game_id', 'original_event_id', 'period_id', 'time_seconds', 'pass_direction']],
    on=['game_id', 'original_event_id', 'period_id', 'time_seconds'],
    how='left'
)
df_actions['pass_direction'] = df_actions['pass_direction'].fillna('not_a_pass')

print("Merged 'pass_direction' into df_actions and filled NaN values. Displaying head of df_actions:")
df_actions.head()

## Analyze xT Contribution of Forward Passes

### Subtask:
Filter df_actions to isolate only 'pass' actions. Then, analyze the xT contribution of 'forward passes' by calculating descriptive statistics (mean, median, sum) of their 'xT_value'.


**Reasoning**:
To analyze the xT contribution of forward passes, I will filter the `df_passes` DataFrame to include only passes with a 'forward' direction, then calculate the mean, median, and sum of their 'xT_value'.



In [ ]:
forward_passes = df_passes[df_passes['pass_direction'] == 'forward'].copy()

mean_xt_forward_passes = forward_passes['xT_value'].mean()
median_xt_forward_passes = forward_passes['xT_value'].median()
sum_xt_forward_passes = forward_passes['xT_value'].sum()

print(f"Mean xT of forward passes: {mean_xt_forward_passes:.4f}")
print(f"Median xT of forward passes: {median_xt_forward_passes:.4f}")
print(f"Sum of xT for all forward passes: {sum_xt_forward_passes:.4f}")

## Compare xT Across Pass Directions

### Subtask:
Compare the average xT generated by 'forward passes', 'backward passes', and 'sideways passes' to understand their relative contributions to offensive threat.


**Reasoning**:
To compare the average xT for each pass direction, I will group the `df_passes` DataFrame by 'pass_direction' and calculate the mean of 'xT_value' for each group.



In [ ]:
avg_xt_by_direction = df_passes.groupby('pass_direction')['xT_value'].mean()

print("Average xT by Pass Direction:")
print(avg_xt_by_direction)

## Visualize Forward Pass xT Distribution

### Subtask:
Create visualizations (e.g., histogram or box plot) to show the distribution of xT values specifically for forward passes, providing insight into their typical threat range.


**Reasoning**:
To visualize the distribution of xT values for forward passes, a histogram is suitable. I will use the `forward_passes` DataFrame and plot the `xT_value` column, adding appropriate labels and a title.



In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(forward_passes['xT_value'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of xT Values for Forward Passes')
plt.xlabel('xT Value')
plt.ylabel('Frequency')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

print("Visualized the distribution of xT values for forward passes using a histogram.")

## Final Task

### Subtask:
Summarize the findings regarding the xT contribution of forward passes, including comparisons with other pass types and the overall significance of forward passes in generating offensive threat.


## Summary:

### Q&A
The task asked for a summary of the findings regarding the xT contribution of forward passes, including comparisons with other pass types and the overall significance of forward passes in generating offensive threat.

*   **xT contribution of forward passes**: Forward passes have a mean xT of 0.0007 and a median xT of 0.0000. Their cumulative xT across all analyzed passes is 0.8637.
*   **Comparison with other pass types**: Forward passes (average xT: 0.000706) generate less average offensive threat than sideways passes (average xT: 0.000788), but significantly more than backward passes (average xT: -0.000191).
*   **Overall significance of forward passes in generating offensive threat**: While individual forward passes often have a low or zero xT value, their overall cumulative contribution to offensive threat is positive, indicating they are a crucial component of progressing the ball and creating danger, though not always the highest individual threat generators compared to sideways passes.

### Data Analysis Key Findings
*   Passes were categorized into 'forward', 'backward', or 'sideways' based on the change in their 'start_x' and 'end_x' coordinates.
*   Forward passes have an average xT value of approximately 0.0007.
*   The median xT value for forward passes is 0.0000, suggesting that many individual forward passes contribute very little or no expected threat.
*   The total cumulative xT from all forward passes analyzed is 0.8637, indicating their significant collective contribution to offensive threat.
*   Sideways passes exhibit the highest average xT among all pass directions, with an average of 0.000788.
*   Forward passes rank second in average xT (0.000706), closely behind sideways passes.
*   Backward passes have a slightly negative average xT of -0.000191, implying they tend to marginally decrease offensive threat.
*   The distribution of xT values for forward passes indicates that while most passes have xT values close to zero, there are some forward passes that generate substantially higher expected threat.

### Insights or Next Steps
*   Despite a low median xT, the positive mean and significant sum of xT for forward passes underscore their collective importance in overall offensive progression, even if they don't always directly create high-threat situations.
*   Investigate the characteristics (e.g., length, location, recipient) of sideways passes that lead to their slightly higher average xT compared to forward passes, as this could reveal valuable tactical insights.


# Task
Define a function to categorize the `start_x` and `end_x` coordinates of each pass into discrete pitch zones (e.g., defensive zone, midfield zone, attacking zone) and apply this function to `df_passes` to create new `start_zone` and `end_zone` columns.

## Define Pitch Zones

### Subtask:
Create a function to categorize the `start_x` and `end_x` coordinates of each pass into discrete 'zones' (e.g., defensive zone, midfield zone, attacking zone) and apply this function to `df_passes` to create new `start_zone` and `end_zone` columns.


**Reasoning**:
First, I will define a function `get_pitch_zone` to categorize x-coordinates into defensive, midfield, and attacking zones. Then, I will apply this function to the `start_x` and `end_x` columns of `df_passes` to create `start_zone` and `end_zone` columns, respectively. Finally, I will display the head of the updated DataFrame.



In [ ]:
def get_pitch_zone(x_coordinate):
    if 0 <= x_coordinate < spadl.config.field_length / 3:
        return 'defensive_zone'
    elif spadl.config.field_length / 3 <= x_coordinate < 2 * spadl.config.field_length / 3:
        return 'midfield_zone'
    elif 2 * spadl.config.field_length / 3 <= x_coordinate <= spadl.config.field_length:
        return 'attacking_zone'
    else:
        return 'out_of_bounds' # Should ideally not happen with valid data

df_passes['start_zone'] = df_passes['start_x'].apply(get_pitch_zone)
df_passes['end_zone'] = df_passes['end_x'].apply(get_pitch_zone)

print("Added 'start_zone' and 'end_zone' to df_passes. Displaying head:")
df_passes.head()

## Filter for Forward Passes

### Subtask:
Isolate only `forward` passes from `df_passes`, which already contains the `pass_direction` column.


**Reasoning**:
To isolate forward passes, I will filter the `df_passes` DataFrame where the 'pass_direction' is 'forward' and store the result in a new DataFrame called `forward_passes`.



In [ ]:
forward_passes = df_passes[df_passes['pass_direction'] == 'forward'].copy()
print("Filtered `df_passes` to isolate 'forward' passes. Displaying head of `forward_passes`:")
forward_passes.head()

## Analyze xT by Zone and Success

### Subtask:
Group the `forward_passes` DataFrame by `start_zone`, `end_zone`, and `result_name` and calculate the mean `xT_value` for each group.


**Reasoning**:
To analyze the xT contribution of forward passes based on their start zone, end zone, and result, I will group the 'forward_passes' DataFrame by these columns and calculate the mean of 'xT_value' for each group.



In [ ]:
xt_by_zone_success = forward_passes.groupby(['start_zone', 'end_zone', 'result_name'])['xT_value'].mean().reset_index()

print("Mean xT value for forward passes by start zone, end zone, and result:")
print(xt_by_zone_success)

## Visualize xT by Zone for Forward Passes

### Subtask:
Create visualizations (e.g., grouped bar charts) to show the average xT of successful and failed forward passes based on their starting and ending zones. This will highlight which regional forward passes are most valuable when successful, and which unsuccessful ones lose the most xT.


**Reasoning**:
To visualize the average xT of successful and failed forward passes by zone, I will first pivot the `xt_by_zone_success` DataFrame to have 'result_name' as columns ('success' and 'fail') and then create a grouped bar chart. I'll also combine 'start_zone' and 'end_zone' into a single 'zone_transition' column for better plotting.



In [ ]:
import matplotlib.pyplot as plt

xt_by_zone_success['zone_transition'] = xt_by_zone_success['start_zone'] + ' to ' + xt_by_zone_success['end_zone']

pivoted_xt_by_zone = xt_by_zone_success.pivot_table(
    index='zone_transition',
    columns='result_name',
    values='xT_value',
    fill_value=0
).reset_index()

pivoted_xt_by_zone.columns.name = None

# Filter for 'success' and 'fail' columns, as 'offside' might exist but is not explicitly requested for comparison
plot_data = pivoted_xt_by_zone[['zone_transition', 'success', 'fail']]

plot_data.plot(x='zone_transition', y=['success', 'fail'], kind='bar', figsize=(15, 7), width=0.8)
plt.title('Average xT of Forward Passes by Zone Transition and Success')
plt.xlabel('Zone Transition')
plt.ylabel('Average xT Value')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("Visualized the average xT of successful and failed forward passes by zone transition.")

## Final Task

### Subtask:
Summarize the findings regarding the xT contribution of forward passes, including comparisons with other pass types and the overall significance of forward passes in generating offensive threat.


## Summary:

### Q&A
The analysis focused on summarizing the findings regarding the xT contribution of forward passes and their significance in generating offensive threat. Comparisons with other pass types were not performed in this analysis.

### Data Analysis Key Findings
*   **Zone Categorization**: `df_passes` was successfully augmented with `start_zone` and `end_zone` columns, categorizing pass locations into 'defensive\_zone', 'midfield\_zone', and 'attacking\_zone' based on x-coordinates.
*   **Forward Pass xT Values**: Failed forward passes and those resulting in 'offside' consistently showed an average xT value of $0.000000$, indicating no threat generation.
*   **Most Threatening Forward Passes**: Successful forward passes significantly increase offensive threat, especially those progressing the ball into more advanced areas.
    *   Successful forward passes from the 'defensive\_zone' to the 'attacking\_zone' yielded the highest average xT of $0.008106$.
    *   Successful forward passes from the 'midfield\_zone' to the 'attacking\_zone' had an average xT of $0.003001$.
    *   Successful forward passes from the 'defensive\_zone' to the 'midfield\_zone' showed an average xT of $0.001864$.
*   **Intra-Attacking Zone Passes**: Successful forward passes remaining within the 'attacking\_zone' (attacking\_zone to attacking\_zone) had a slightly negative average xT of $-0.001406$, suggesting that some short passes within this zone might minimally reduce immediate threat.

### Insights or Next Steps
*   **Strategic Pass Prioritization**: Teams should prioritize training and execution of long forward passes from defensive and midfield zones directly into the attacking zone, as these yielded the highest xT values upon success.
*   **Contextualizing Attacking Zone Passes**: Further investigation is needed to understand why successful forward passes within the attacking zone sometimes show negative xT, potentially by analyzing pass direction (e.g., sideways, backward within the zone) or proximity to goal/defenders, to inform decision-making in critical areas.


# Task
Filter `df_actions` to isolate only `pass` actions and identify the immediately subsequent action by the same team in the same game and period. Store these subsequent actions' xT values with their corresponding passes in a new DataFrame called `df_passes_with_next_action_xt`.

## Identify Passes and Next Actions

### Subtask:
Filter `df_actions` to get all pass actions. For each pass, identify the immediately subsequent action by the same team in the same game and period. Store these subsequent actions' xT values with their corresponding passes in a new DataFrame called `df_passes_with_next_action_xt`.


**Reasoning**:
First, I will sort `df_actions` to ensure correct sequential processing, then filter for 'pass' actions to create `df_passes_with_next_action_xt`. Subsequently, I will add shifted columns for game_id, period_id, team_id, and xT_value to `df_actions` to identify the properties of the next action. Finally, I will merge these shifted values back into `df_passes_with_next_action_xt`, ensuring the next action is by the same team in the same game and period, and rename the xT column.



In [ ]:
df_actions_sorted = df_actions.sort_values(by=['game_id', 'period_id', 'time_seconds']).reset_index(drop=True)
df_passes_with_next_action_xt = df_actions_sorted[df_actions_sorted['type_name'] == 'pass'].copy()

df_actions_shifted = df_actions_sorted.copy()
df_actions_shifted['next_game_id'] = df_actions_shifted['game_id'].shift(-1)
df_actions_shifted['next_period_id'] = df_actions_shifted['period_id'].shift(-1)
df_actions_shifted['next_team_id'] = df_actions_shifted['team_id'].shift(-1)
df_actions_shifted['next_action_xT'] = df_actions_shifted['xT_value'].shift(-1)

df_passes_with_next_action_xt = df_passes_with_next_action_xt.merge(
    df_actions_shifted[['game_id', 'period_id', 'time_seconds', 'next_game_id', 'next_period_id', 'next_team_id', 'next_action_xT']],
    on=['game_id', 'period_id', 'time_seconds'],
    how='left'
)

# Filter for cases where the next action is by the same team in the same game and period
condition = (
    (df_passes_with_next_action_xt['game_id'] == df_passes_with_next_action_xt['next_game_id']) &
    (df_passes_with_next_action_xt['period_id'] == df_passes_with_next_action_xt['next_period_id']) &
    (df_passes_with_next_action_xt['team_id'] == df_passes_with_next_action_xt['next_team_id'])
)
df_passes_with_next_action_xt.loc[~condition, 'next_action_xT'] = np.nan

# Drop the temporary next_* columns
df_passes_with_next_action_xt = df_passes_with_next_action_xt.drop(columns=[
    'next_game_id', 'next_period_id', 'next_team_id'
])

print("Created df_passes_with_next_action_xt with the xT value of the subsequent action. Displaying head:")
df_passes_with_next_action_xt.head()

## Calculate xT Change

### Subtask:
Calculate the change in xT (next action's xT - pass's xT) for each pass that has a subsequent action. Store this in a new column called `xT_change`.


**Reasoning**:
I will create a new column 'xT_change' by subtracting the 'xT_value' from 'next_action_xT' in the `df_passes_with_next_action_xt` DataFrame to calculate the change in expected threat.



In [ ]:
df_passes_with_next_action_xt['xT_change'] = df_passes_with_next_action_xt['next_action_xT'] - df_passes_with_next_action_xt['xT_value']

print("Calculated 'xT_change' and added it to df_passes_with_next_action_xt. Displaying head:")
df_passes_with_next_action_xt.head()

## Analyze xT Change by Pass Result

### Subtask:
Group the passes by their `result_name` ('success' or 'fail') and calculate descriptive statistics (mean, median, standard deviation) of the `xT_change` for each group. This will show how successful vs. failed passes affect the subsequent xT.


**Reasoning**:
To analyze the xT change by pass result, I will group the DataFrame by 'result_name' and then calculate the mean, median, and standard deviation of 'xT_change' for each group.



In [ ]:
xt_change_stats = df_passes_with_next_action_xt.groupby('result_name')['xT_change'].agg(['mean', 'median', 'std'])

print("Descriptive statistics of xT_change by pass result:")
print(xt_change_stats)

## Visualize xT Change Distribution

### Subtask:
Create visualizations (e.g., histograms or box plots) to compare the distribution of `xT_change` for successful passes versus failed passes. This will visually highlight the impact of pass outcome on the subsequent offensive threat.


**Reasoning**:
To compare the distribution of `xT_change` for successful and failed passes, I will first filter the `df_passes_with_next_action_xt` DataFrame to create two new DataFrames, one for successful passes and one for failed passes. Then, I will create and display a histogram for the `xT_change` column for each of these filtered DataFrames.



In [ ]:
successful_passes_xt_change = df_passes_with_next_action_xt[df_passes_with_next_action_xt['result_name'] == 'success']['xT_change'].dropna()
failed_passes_xt_change = df_passes_with_next_action_xt[df_passes_with_next_action_xt['result_name'] == 'fail']['xT_change'].dropna()

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.hist(successful_passes_xt_change, bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of xT Change for Successful Passes')
plt.xlabel('xT Change')
plt.ylabel('Frequency')
plt.grid(True, linestyle='--', alpha=0.6)

plt.subplot(1, 2, 2)
plt.hist(failed_passes_xt_change, bins=50, edgecolor='black', alpha=0.7, color='salmon')
plt.title('Distribution of xT Change for Failed Passes')
plt.xlabel('xT Change')
plt.ylabel('Frequency')
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

print("Visualized the distribution of xT change for successful and failed passes using histograms.")

## Final Task

### Subtask:
Summarize the findings regarding how successful and failed passes influence the xT of subsequent actions, drawing conclusions about the importance of pass outcomes in offensive threat generation.


## Summary:

### Q&A
Successful passes lead to a slight increase in the expected threat (xT) of subsequent actions, while failed passes generally result in a decrease in xT. This indicates that pass outcomes are crucial for maintaining and advancing offensive threat in a game.

### Data Analysis Key Findings
*   A DataFrame `df_passes_with_next_action_xt` was created, capturing all pass actions along with the xT value of the immediately subsequent action by the same team in the same game and period. If no such action followed, the `next_action_xT` was recorded as `NaN`.
*   A new column, `xT_change`, was calculated as the difference between the `next_action_xT` and the pass's `xT_value`, quantifying the change in offensive threat.
*   **Successful passes** demonstrated a positive, albeit small, average `xT_change` of approximately 0.000523, with a median of 0.000176, indicating a tendency to slightly increase the offensive threat. The standard deviation was 0.020471.
*   **Failed passes** showed a negative average `xT_change` of approximately -0.005656, with a median of 0, suggesting a decrease in the expected threat. The standard deviation was 0.029093.
*   The `offside` category did not have any associated `xT_change` data, resulting in `NaN` statistics.
*   Visualizations (histograms) confirmed these findings, showing distinct distributions of `xT_change` for successful versus failed passes, with successful passes generally shifting the distribution towards positive xT changes and failed passes towards negative xT changes.

### Insights or Next Steps
*   **Insight**: Successful passes, despite a small average increase, are fundamental for positive offensive progression, while failed passes actively diminish the team's immediate offensive threat. The difference in mean `xT_change` between successful and failed passes (approximately 0.006) highlights the value of pass accuracy.
*   **Next Steps**: Investigate the specific types of actions that follow successful vs. failed passes to understand how different subsequent actions contribute to or detract from xT. This could involve analyzing the `type_name` of the `next_action`.
